In [23]:
import logging
from naslib.defaults.trainer import Trainer
from naslib.optimizers import DARTSOptimizer
from naslib.search_spaces import DartsSearchSpace
from naslib.utils import utils, setup_logger, get_config_from_args, set_seed, log_args
from naslib.search_spaces.core.graph import Graph, EdgeData
from naslib.search_spaces.core import primitives as ops
from torch import nn
from fvcore.common.config import CfgNode
from copy import deepcopy
from IPython.display import clear_output
import torch
from naslib.search_spaces.core.primitives import AbstractPrimitive
from activation_sub_func.binary_func import *
from activation_sub_func.unary_func import *

In [24]:
config = utils.get_config_from_args(config_type='nas')
config.optimizer = 'darts' # 'gdas', 'drnas'
config.search.batch_size = 16
config.search.learning_rate = 0.00002
config.search.arch_learning_rate = 0.000003
utils.set_seed(config.seed)
clear_output(wait=True)
utils.log_args(config)

logger = setup_logger(config.save + '/log.log')
logger.setLevel(logging.INFO)

[07/14 18:52:57 nl.utils.utils]: dataset....................................cifar10
[07/14 18:52:57 nl.utils.utils]: seed.............................................0
[07/14 18:52:57 nl.utils.utils]: search_space...........................nasbench201
[07/14 18:52:57 nl.utils.utils]: out_dir........................................run
[07/14 18:52:57 nl.utils.utils]: optimizer....................................darts
[07/14 18:52:57 nl.utils.utils]: searchacq_fn_optimization: random_sampling
acq_fn_type: its
arch_learning_rate: 3e-06
arch_weight_decay: 0.001
batch_size: 16
checkpoint_freq: 1000
cutout: False
cutout_length: 16
cutout_prob: 1.0
data_size: 25000
debug_predictor: False
drop_path_prob: 0.0
encoding_type: adjacency_one_hot
epochs: 100
fidelity: -1
gpu: None
grad_clip: 5
k: 10
learning_rate: 2e-05
learning_rate_min: 0.001
max_mutations: 1
momentum: 0.9
num_arches_to_mutate: 2
num_candidates: 20
num_ensemble: 3
num_init: 10
output_weights: True
population_size: 30
predictor_typ

In [25]:
class ConvNetSearchSpace(Graph):

    OPTIMIZER_SCOPE = [
        'activation_1'
    ]

    QUERYABLE = False

    def __init__(self):
        super().__init__()

        # cell definition
        activation_cell = Graph()
        activation_cell.name = 'activation_cell'
        activation_cell.add_node(1)  # input node
        activation_cell.add_node(2)  # unary node / intermediate node
        activation_cell.add_node(3)  # unary node / intermediate node
        activation_cell.add_node(4)  # binary node / output node
        activation_cell.add_edges_from([(1, 2, EdgeData())])  # mutable intermediate edge
        activation_cell.edges[1, 2].set('cell_name', 'activation_cell')
        activation_cell.add_edges_from([(1, 3, EdgeData())])  # mutable intermediate edge
        activation_cell.edges[1, 3].set('cell_name', 'activation_cell')

        activation_cell.add_edges_from([(2, 4, EdgeData().finalize())])  # mutable intermediate edge
        activation_cell.add_edges_from([(3, 4, EdgeData().finalize())])  # mutable intermediate edge
        activation_cell.nodes[4]['comb_op'] = Stack()

        activation_cell.add_node(5)  # binary node
        activation_cell.add_edges_from([(4, 5, EdgeData())])  # mutable intermediate edge
        activation_cell.edges[4, 5].set('cell_name', 'activation_cell')
        
        activation_cell.add_node(6)
        activation_cell.add_edges_from([(5, 6, EdgeData().finalize())])
        
        # macroarchitecture definition
        self.name = 'makrograph'
        self.add_node(1) # input node
        self.add_node(2) # intermediate node
        self.add_node(3, subgraph=activation_cell.copy().set_scope('activation_1').set_input([2])) # activation cell
        self.nodes[3]['subgraph'].name = 'activation_1'
        self.add_node(4) # output node
        self.add_edges_from([(i, i+1, EdgeData()) for i in range(1, 4)])
        self.edges[1, 2].set('op',
            ops.Sequential(
                nn.Conv2d(3, 6, 5),
                nn.MaxPool2d(2),
                nn.Conv2d(6, 16, 5),
                nn.MaxPool2d(2),
                nn.Flatten()
            )) # convolutional edge
        self.edges[3, 4].set('op', 
            ops.Sequential(
                nn.Linear(400, 10), 
                nn.Softmax(dim=1)
            )) # linear edge
        
        self.update_edges(
            update_func=lambda edge: self._set_ops(edge),
            scope='activation_1',
            private_edge_data=True,
        )

    def _set_ops(self, edge, channels=32):
        # unary
        if (edge.head, edge.tail) in {(1, 2), (1, 3)}:
            edge.data.set("op", [
                ops.Identity(),
                ops.Zero(stride=1),
#                 Power(2),
#                 Power(3),
                Sqrt(),
#                 Sin(),
#                 Cos(),
                Abs_op(),
                Sign(),
#                 Beta_mul(channels=channels),
#                 Beta_add(channels=channels),
                Log(),
#                 Exp(),
#                 Sinh(),
#                 Cosh(),
                Tanh(),
                Asinh(),
                Atan(),
                Sinc(),
                Maximum0(),
                Minimum0(),
                Sigmoid(),
#                 LogExp(),
#                 Exp2(),
                Erf(),
#                 Beta(channels=channels),
            ])
        # binary
        elif (edge.head, edge.tail) in {(4, 5)}:
            edge.data.set("op", [
                Add(),
                Sub(),
                Mul(),
                Div(),
                Maximum(),
                Minimum(),
                SigMul(),
#                 ExpBetaSub2(channels=channels),
#                 ExpBetaSubAbs(channels=channels),
#                 BetaMix(channels=channels),
            ]) 

In [52]:
class ResNet20SearchSpace(Graph):
    """
    https://www.researchgate.net/figure/ResNet-20-architecture_fig3_351046093
    """

    OPTIMIZER_SCOPE = [
        f"activation_{i}" for i in range(1, 20)
    ]

    QUERYABLE = False

    def __init__(self):
        super().__init__()

        # cell definition
#         activation_cell = Graph()
#         activation_cell.name = 'activation_cell'
#         activation_cell.add_node(1)  # input node
#         activation_cell.add_node(2)  # unary node / intermediate node
#         activation_cell.add_node(3)  # unary node / intermediate node
#         activation_cell.add_node(4)  # binary node / output node
#         activation_cell.add_edges_from([(1, 2, EdgeData())])  # mutable intermediate edge
#         activation_cell.edges[1, 2].set('cell_name', 'activation_cell')
#         activation_cell.add_edges_from([(1, 3, EdgeData())])  # mutable intermediate edge
#         activation_cell.edges[1, 3].set('cell_name', 'activation_cell')

#         activation_cell.add_edges_from([(2, 4, EdgeData().finalize())])  # mutable intermediate edge
#         activation_cell.add_edges_from([(3, 4, EdgeData().finalize())])  # mutable intermediate edge
#         activation_cell.nodes[4]['comb_op'] = Stack()

#         activation_cell.add_node(5)  # binary node
#         activation_cell.add_edges_from([(4, 5, EdgeData())])  # mutable intermediate edge
#         activation_cell.edges[4, 5].set('cell_name', 'activation_cell')
        
#         activation_cell.add_node(6)
#         activation_cell.add_edges_from([(5, 6, EdgeData().finalize())])

          # linear activation (testing)
        activation_cell = Graph()
        activation_cell.name = 'activation_cell'
        activation_cell.add_node(1)  # input node
        activation_cell.add_node(2)  # unary node / intermediate node
        activation_cell.add_node(3)  # unary node / intermediate node
        activation_cell.add_edges_from([(1, 2, EdgeData().finalize())])  # immutable intermediate edge
        activation_cell.add_edges_from([(2, 3, EdgeData())])  # immutable intermediate edge

        # macroarchitecture definition
        self.name = 'makrograph'
        self.add_node(1)  # input
        self.add_node(2)  # intermediate
        self.add_node(3,
                      subgraph=activation_cell.copy().set_scope("activation_1").set_input([2]))  # activation cell 3
        self.nodes[3]['subgraph'].name = "activation_1"
        self.update_edges(
            update_func=lambda edge: self._set_ops(edge, 16),
            scope=f"activation_{1}",
            private_edge_data=True, )

        self.add_node(4)
        self.add_node(5,
                      subgraph=activation_cell.copy().set_scope("activation_2").set_input([4]))  # activation cell 3
        self.nodes[5]['subgraph'].name = "activation_2"
        self.update_edges(
            update_func=lambda edge: self._set_ops(edge, 16),
            scope=f"activation_{2}",
            private_edge_data=True, )

        self.add_node(6)
        # Todo add option here with a func which has a arg channels
        self.add_node(7,
                      subgraph=activation_cell.copy().set_scope("activation_3").set_input([6]))  # activation cell 3
        self.nodes[7]['subgraph'].name = "activation_3"
        self.update_edges(
            update_func=lambda edge: self._set_ops(edge, 16),
            scope=f"activation_{3}",
            private_edge_data=True, )

        self.add_edges_from([
            (1, 2, EdgeData()),
            (2, 3, EdgeData()),
            (3, 4, EdgeData()),
            (4, 5, EdgeData()),
            (5, 6, EdgeData()),
            (3, 6, EdgeData()),
            (6, 7, EdgeData())
        ])

        self.edges[1, 2].set('op',
                             ops.Sequential(nn.Conv2d(3, 16, 3, padding=1), ))  # convolutional edge
        self.edges[3, 4].set('op',
                             ops.Sequential(nn.Conv2d(16, 16, 3, padding=1), ))  # convolutional edge
        self.edges[5, 6].set('op',
                             ops.Sequential(nn.Conv2d(16, 16, 3, padding=1), ))  # convolutional edge

        conv_option = {
            "in_channels": 16,
            "out_channels": 16,
            "kernel_size": 3,
            "padding": 1
        }
        self._create_base_block(7, 4, activation_cell, conv_option)
        self._create_base_block(11, 6, activation_cell, conv_option)

        conv_option_a = {
            "in_channels": 16,
            "out_channels": 32,
            "kernel_size": 3,
            "padding": 1,
            "stride": 2
        }
        conv_option_b = {
            "in_channels": 16,
            "out_channels": 32,
            "kernel_size": 1,
            "padding": 0,
            "stride": 2
        }
        self._create_reduction_block(15, 8, activation_cell, conv_option_a, conv_option_b)

        conv_option = {
            "in_channels": 32,
            "out_channels": 32,
            "kernel_size": 3,
            "padding": 1
        }
        self._create_base_block(19, 10, activation_cell, conv_option)
        self._create_base_block(23, 12, activation_cell, conv_option)

        conv_option_a = {
            "in_channels": 32,
            "out_channels": 64,
            "kernel_size": 3,
            "padding": 1,
            "stride": 2
        }
        conv_option_b = {
            "in_channels": 32,
            "out_channels": 64,
            "kernel_size": 1,
            "padding": 0,
            "stride": 2
        }
        self._create_reduction_block(27, 14, activation_cell, conv_option_a, conv_option_b)

        conv_option = {
            "in_channels": 64,
            "out_channels": 64,
            "kernel_size": 3,
            "padding": 1
        }
        self._create_base_block(31, 16, activation_cell, conv_option)
        self._create_base_block(34, 18, activation_cell, conv_option)

        # add head
        self.add_node(39)
        self.add_edges_from([
            (38, 39, EdgeData())
        ])
        self.edges[38, 39].set('op',
                               ops.Sequential(
                                   nn.AvgPool2d(8),
                                   nn.Flatten(),
                                   nn.Linear(64, 10),
                                   nn.Softmax(dim=1)
                               ))  # convolutional edge
        self.add_node(40)
        self.add_edges_from([
            (39, 40, EdgeData().finalize())
        ])

    def _create_base_block(self, start: int, stage: int, cell, conv_option: dict):
        self.add_node(start + 1)

        self.add_node(start + 2, subgraph=cell.copy().set_scope(f"activation_{stage}").set_input(
            [start + 1]))  # activation cell 3
        self.nodes[start + 2]['subgraph'].name = f"activation_{stage}"
        self.update_edges(
            update_func=lambda edge: self._set_ops(edge, conv_option["out_channels"]),
            scope=f"activation_{stage}",
            private_edge_data=True, )

        self.add_node(start + 3)

        self.add_node(start + 4, subgraph=cell.copy().set_scope(f"activation_{stage + 1}").set_input(
            [start + 3]))  # activation cell 3
        self.nodes[start + 4]['subgraph'].name = f"activation_{stage + 1}"
        self.update_edges(
            update_func=lambda edge: self._set_ops(edge, conv_option["out_channels"]),
            scope=f"activation_{stage + 1}",
            private_edge_data=True, )

        self.add_edges_from([
            (start, start + 1, EdgeData()),
            (start, start + 3, EdgeData()),
            (start + 1, start + 2, EdgeData()),
            (start + 2, start + 3, EdgeData()),
            (start + 3, start + 4, EdgeData()),
        ])

        self.edges[start, start + 1].set('op',
                                         ops.Sequential(nn.Conv2d(**conv_option), ))  # convolutional edge
        self.edges[start + 2, start + 3].set('op',
                                             ops.Sequential(nn.Conv2d(**conv_option), ))  # convolutional edge

    def _create_reduction_block(self, start: int, stage: int, cell, conv_option_a: dict, conv_option_b: dict):
        self.add_node(start + 1)

        self.add_node(start + 2, subgraph=cell.copy().set_scope(f"activation_{stage}").set_input(
            [start + 1]))  # activation cell 3
        self.nodes[start + 2]['subgraph'].name = f"activation_{stage}"
        self.update_edges(
            update_func=lambda edge: self._set_ops(edge, conv_option_a["out_channels"]),
            scope=f"activation_{stage}",
            private_edge_data=True, )

        self.add_node(start + 3)

        self.add_node(start + 4, subgraph=cell.copy().set_scope(f"activation_{stage + 1}").set_input(
            [start + 3]))  # activation cell 3
        self.nodes[start + 4]['subgraph'].name = f"activation_{stage + 1}"
        self.update_edges(
            update_func=lambda edge: self._set_ops(edge, conv_option_b["out_channels"]),
            scope=f"activation_{stage + 1}",
            private_edge_data=True, )

        self.add_edges_from([
            (start, start + 1, EdgeData()),
            (start, start + 3, EdgeData()),  # add conv
            (start + 1, start + 2, EdgeData()),
            (start + 2, start + 3, EdgeData()),
            (start + 3, start + 4, EdgeData()),
        ])

        self.edges[start, start + 1].set('op',
                                         ops.Sequential(nn.Conv2d(**conv_option_a), ))  # convolutional edge
        conv_option_a["in_channels"] = conv_option_a["out_channels"]
        conv_option_a["stride"] = 1

        self.edges[start, start + 3].set('op',
                                         ops.Sequential(nn.Conv2d(**conv_option_b), ))  # convolutional edge
        self.edges[start + 2, start + 3].set('op',
                                             ops.Sequential(nn.Conv2d(**conv_option_a), ))  # convolutional edge

    def _set_ops(self, edge, channels=32):
        # unary
        if (edge.head, edge.tail) in {(1, 2), (1, 3)}:
            edge.data.set("op", [
                ops.Identity(),
                ops.Zero(stride=1),
#                 Power(2),
#                 Power(3),
#                 Sqrt(),
#                 Sin(),
#                 Cos(),
#                 Abs_op(),
#                 Sign(),
#                 Beta_mul(channels=channels),
#                 Beta_add(channels=channels),
#                 Log(),
#                 Exp(),
#                 Sinh(),
#                 Cosh(),
#                 Tanh(),
#                 Asinh(),
#                 Atan(),
#                 Sinc(),
#                 Maximum0(),
#                 Minimum0(),
#                 Sigmoid(),
#                 LogExp(),
#                 Exp2(),
#                 Erf(),
#                 Beta(channels=channels),
            ])
        # binary
        elif (edge.head, edge.tail) in {(4, 5)}:
            edge.data.set("op", [
#                 Add(),
#                 Sub(),
#                 Mul(),
#                 Div(),
                Maximum(),
#                 Minimum(),
#                 SigMul(),
#                 ExpBetaSub2(channels=channels),
#                 ExpBetaSubAbs(channels=channels),
#                 BetaMix(channels=channels),
            ]) 
        # testing
        elif (edge.head, edge.tail) in {(2, 3)}:
            edge.data.set("op", [
                  ops.Identity()
            ])

In [53]:
search_space = ResNet20SearchSpace()

In [54]:
optimizer = DARTSOptimizer(config)
optimizer.adapt_search_space(search_space)

[07/14 19:02:48 nl.optimizers.oneshot.darts.optimizer]: Parsed graph:
Graph activation_1:
 Graph(
  (activation_1-edge(1,2)): Identity()
  (activation_1-edge(2,3)): MixedOp(
    (primitive-0): Identity()
  )
)
Graph activation_10:
 Graph(
  (activation_10-edge(1,2)): Identity()
  (activation_10-edge(2,3)): MixedOp(
    (primitive-0): Identity()
  )
)
Graph activation_11:
 Graph(
  (activation_11-edge(1,2)): Identity()
  (activation_11-edge(2,3)): MixedOp(
    (primitive-0): Identity()
  )
)
Graph activation_12:
 Graph(
  (activation_12-edge(1,2)): Identity()
  (activation_12-edge(2,3)): MixedOp(
    (primitive-0): Identity()
  )
)
Graph activation_13:
 Graph(
  (activation_13-edge(1,2)): Identity()
  (activation_13-edge(2,3)): MixedOp(
    (primitive-0): Identity()
  )
)
Graph activation_14:
 Graph(
  (activation_14-edge(1,2)): Identity()
  (activation_14-edge(2,3)): MixedOp(
    (primitive-0): Identity()
  )
)
Graph activation_15:
 Graph(
  (activation_15-edge(1,2)): Identity()
  (act

In [ ]:
trainer = Trainer(optimizer, config)
trainer.search()

[07/14 19:02:51 nl.defaults.trainer]: param size = 0.271690MB
[07/14 19:02:51 nl.defaults.trainer]: Start training
Files already downloaded and verified
Files already downloaded and verified
[07/14 19:02:52 nl.optimizers.oneshot.darts.optimizer]: Arch weights (alphas, last column argmax): 
+0.000323, 0
+0.000323, 0
+0.000323, 0
+0.000323, 0
+0.000323, 0
+0.000323, 0
+0.000323, 0
+0.000323, 0
+0.000323, 0
+0.000323, 0
+0.000323, 0
+0.000323, 0
+0.000323, 0
+0.000323, 0
+0.000323, 0
+0.000323, 0
+0.000323, 0
+0.000323, 0
+0.000323, 0
[07/14 19:02:52 nl.defaults.trainer]: Epoch 0-0, Train loss: 2.30280, validation loss: 2.30030, learning rate: [2e-05]
[07/14 19:02:57 nl.defaults.trainer]: Epoch 0-82, Train loss: 2.30537, validation loss: 2.30619, learning rate: [2e-05]
[07/14 19:03:02 nl.defaults.trainer]: Epoch 0-163, Train loss: 2.29887, validation loss: 2.29970, learning rate: [2e-05]
[07/14 19:03:07 nl.defaults.trainer]: Epoch 0-245, Train loss: 2.29870, validation loss: 2.30630, lear

KeyboardInterrupt: 

In [ ]:
trainer.evaluate()